TO DO:

IMPORTERE:
- importere all dataen: DONE
- remove duplicates: DONE

VISUALIZATION:
- images 
- malignant, benign: DONE
- age
- gender: DONE
- affected areas 
- mean and std?? 

PREPROCESSING:
- make train, validation, and test set
- resize
- rescale

DATA AUGMENTATION:
- random transformation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from glob import glob
import matplotlib.pyplot as plt

from keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
import cv2
import glob
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.python.keras import backend as K
import plotly.graph_objects as go
import plotly.offline as py
autosize =False

from plotly.subplots import make_subplots
import plotly.graph_objects as go

%matplotlib inline

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#train_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
#test_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
#train=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
#test=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')
train = pd.read_csv("../input/siim-isic-melanoma-classification/train.csv")
test = pd.read_csv("../input/siim-isic-melanoma-classification/test.csv")
train.head()
test.head()
train['target']

In [ ]:
# remove duplicates
dup=pd.read_csv("../input/siim-list-of-duplicates/2020_Challenge_duplicates.csv")

drop_index_list =[]
for dup_image in dup.ISIC_id_paired:
  for idx,image in enumerate(train.image_name):
    if image == dup_image:
      drop_index_list.append(idx)

print("Number of duplicates in dataset is:", len(drop_index_list))

train.drop(drop_index_list, inplace=True)

In [ ]:
# visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(train.benign_malignant, palette="Set2")
plt.title("Malignant vs. benign")
plt.show()

# sex
fig, ax = plt.subplots(1,2,figsize=(20,5))

sns.countplot(train.sex, palette="Reds_r", ax=ax[0])
ax[0].set_xlabel("")
ax[0].set_title("Gender counts in training set")

sns.countplot(test.sex, palette="Blues_r", ax=ax[1])
ax[1].set_xlabel("")
ax[1].set_title("Gender counts in test set")
plt.show()

# age
fig, ax = plt.subplots(1,2,figsize=(20,5))

sns.countplot(train.age_approx, color="tomato", ax=ax[0])
labels = ax[0].get_xticklabels()
ax[0].set_xticklabels(labels, rotation=90)
ax[0].set_xlabel("Age")
ax[0].set_title("Age distribution in training set")

sns.countplot(test.age_approx, color="skyblue", ax=ax[1])
labels = ax[1].get_xticklabels()
ax[1].set_xticklabels(labels, rotation=90)
ax[1].set_xlabel("Age")
ax[1].set_title("Age distribution in test set")
plt.show()

# age average
print(train.age_approx.mean())
print(test.age_approx.mean())

# affected areas
fig, ax = plt.subplots(1,2,figsize=(20,5))

sns.countplot(train.anatom_site_general_challenge, color="tomato", ax=ax[0])
labels = ax[0].get_xticklabels()
ax[0].set_xticklabels(labels, rotation=90)
ax[0].set_xlabel("")
ax[0].set_title("Affected areas in training set")

sns.countplot(test.anatom_site_general_challenge, color="skyblue", ax=ax[1])
labels = ax[1].get_xticklabels()
ax[1].set_xticklabels(labels, rotation=90)
ax[1].set_xlabel("")
ax[1].set_title("Affected areas in test set")
plt.show()

In [ ]:
# show images
# https://www.kaggle.com/jagdmir/siim-melanoma-classification-modelling/data?fbclid=IwAR18v3usRMR7jyhZEGhANPbMqGuMaG9Lg8DANpygC3fmNkAvq_UnJxncXHI#Exploratory-Data-Analysis

# benign
df_imlist= train[train['target']==0].sample(100)
imgs = train.sample(30).image_name.values

plt.figure(figsize=(8,8))
imlist = []
df_imlist = df_imlist.head(9)
df_imlist = df_imlist.reset_index()
for i,k in enumerate(imgs):
    im = cv2.imread(str('../input/siim-isic-melanoma-classification/jpeg/train/'+k+'.jpg'))
    im = cv2.resize(im, (224,224))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = im.astype(np.float32)/255
    imlist.append(im)
f,ax = plt.subplots(4,4, figsize =(8,8))
for i, im in enumerate(imlist):
    ax[i//4, i%4].imshow(im)
    ax[i//4, i%4].axis('off')

plt.show()

In [ ]:
# show malignant
df_imlist= train[train['target']==1].sample(100)
imgs = train.sample(30).image_name.values

plt.figure(figsize=(8,8))
imlist = []
df_imlist = df_imlist.head(9)
df_imlist = df_imlist.reset_index()
for i,k in enumerate(imgs):
    im = cv2.imread(str('../input/siim-isic-melanoma-classification/jpeg/train/'+k+'.jpg'))
    im = cv2.resize(im, (224,224))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = im.astype(np.float32)/255
    imlist.append(im)
f,ax = plt.subplots(2,8, figsize =(8,8))
for i, im in enumerate(imlist):
    ax[i//4, i%4].imshow(im)
    ax[i//4, i%4].axis('off')

plt.show()

In [ ]:
# RGB distribution
# https://www.kaggle.com/parulpandey/melanoma-classification-eda-starter

df_benign=train[train['target']==0].sample(1)
df_malignant=train[train['target']==1].sample(1)
print(df_benign)

img_dir = str("../input/siim-isic-melanoma-classification/jpeg/train")

f = plt.figure(figsize=(16,8))
f.add_subplot(1,2, 1)

sample_img = str(df_benign['image_name']+'.jpg')
sample_img = sample_img.split(" ")[4]
sample_img = sample_img[:16]
raw_image = plt.imread(os.path.join(img_dir, str(sample_img)))
plt.imshow(raw_image, cmap='gray')
plt.colorbar()
plt.title('Benign Image')
print(f"Image dimensions:  {raw_image.shape[0],raw_image.shape[1]}")
print(f"Maximum pixel value : {raw_image.max():.1f} ; Minimum pixel value:{raw_image.min():.1f}")
print(f"Mean value of the pixels : {raw_image.mean():.1f} ; Standard deviation : {raw_image.std():.1f}")

f.add_subplot(1,2, 2)

#_ = plt.hist(raw_image.ravel(),bins = 256, color = 'orange',)
_ = plt.hist(raw_image[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
_ = plt.xlabel('Intensity Value')
_ = plt.ylabel('Count')
_ = plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.show()

# malignant
f = plt.figure(figsize=(16,8))
f.add_subplot(1,2, 1)

sample_img = str(df_malignant['image_name']+'.jpg')
sample_img = sample_img.split(" ")[4]
sample_img = sample_img[:16]
raw_image = plt.imread(os.path.join(img_dir, str(sample_img)))
plt.imshow(raw_image, cmap='gray')
plt.colorbar()
plt.title('Malignant Image')
print(f"Image dimensions:  {raw_image.shape[0],raw_image.shape[1]}")
print(f"Maximum pixel value : {raw_image.max():.1f} ; Minimum pixel value:{raw_image.min():.1f}")
print(f"Mean value of the pixels : {raw_image.mean():.1f} ; Standard deviation : {raw_image.std():.1f}")

f.add_subplot(1,2, 2)

#_ = plt.hist(raw_image.ravel(),bins = 256, color = 'orange',)
_ = plt.hist(raw_image[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
_ = plt.xlabel('Intensity Value')
_ = plt.ylabel('Count')
_ = plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.show()

In [ ]:
# RGB of all images
# https://www.kaggle.com/datafan07/analysis-of-melanoma-metadata-and-effnet-ensemble?fbclid=IwAR2y49DfaraG5jAqt1sRs6-CYLIhqnuH1kJzTLEaWX7jZhDyhbGyqUKsRWg

img_stats_path = '/kaggle/input/melanoma2020imgtabular'

train_attr = pd.read_csv(
    os.path.join(img_stats_path, 'train_mean_colorres.csv'))
test_attr = pd.read_csv(os.path.join(img_stats_path, 'test_mean_colorres.csv'))

train_attr.head()

In [ ]:
train = pd.concat([train, train_attr], axis=1)
test = pd.concat([test, test_attr], axis=1)

train['res'] = train['width'].astype(str) + 'x' + train['height'].astype(str)
test['res'] = test['width'].astype(str) + 'x' + test['height'].astype(str)

In [ ]:
# Creating a customized chart and giving in figsize etc.
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.style.use('ggplot')

fig = plt.figure(constrained_layout=True, figsize=(20, 12))

# Creating a grid with 3 columns and 3 rows

grid = gridspec.GridSpec(ncols=3, nrows=3, figure=fig)

# Customizing the first grid.

ax1 = fig.add_subplot(grid[0, :2])

# Setting the title.

ax1.axes.set_title("RGB Channels of Benign Images",fontsize=30)

# Setting the axis font size 
ax1.tick_params(labelsize=20)

# Setting the range of the y-axis 
ax1.set(ylim=(0, 0.02))

# Plot.

sns.distplot(train[train['target'] == 0].reds, #Pick out the reds from benign training images (target variable equals 0)
             hist_kws={
                 'rwidth': 0.75,
                 'edgecolor': 'black',
                 'alpha': 0.3
             },
             color='red',
             kde=True,
             ax=ax1,
             label='Reds')
sns.distplot(train[train['target'] == 0].greens, #Pick out the greens 
             hist_kws={
                 'rwidth': 0.75,
                 'edgecolor': 'black',
                 'alpha': 0.3
             },
             color='green',
             kde=True,
             ax=ax1,
             label='Greens')
sns.distplot(train[train['target'] == 0].blues,#Pick out the blues 
             hist_kws={
                 'rwidth': 0.75,
                 'edgecolor': 'black',
                 'alpha': 0.3
             },
             color='blue',
             kde=True,
             ax=ax1,
             label='Blues')

ax1.legend(fontsize='x-large', title_fontsize='60',loc='upper left')
ax1.set_xlabel("Intensity value",fontsize=25)
ax1.set_ylabel("Density",fontsize=25)


# Customizing the second grid.

ax2 = fig.add_subplot(grid[1, :2])


# Set the title.

ax2.axes.set_title("RGB Channels of Malignant Images",fontsize=30)

# Setting the axis font size 
ax2.tick_params(labelsize=20)

# Setting the range of the y-axis 
ax2.set(ylim=(0, 0.02))


# Plot

sns.distplot(train[train['target'] == 1].reds,#Pick out the reds from malignant training images (target variable equals 1)
             hist_kws={
                 'rwidth': 0.75,
                 'edgecolor': 'black',
                 'alpha': 0.3
             },
             color='red',
             kde=True,
             ax=ax2,
             label='Reds')
sns.distplot(train[train['target'] == 1].greens,#Pick out the greens 
             hist_kws={
                 'rwidth': 0.75,
                 'edgecolor': 'black',
                 'alpha': 0.3
             },
             color='green',
             kde=True,
             ax=ax2,
             label='Greens')
sns.distplot(train[train['target'] == 1].blues,#Pick out the blues 
             hist_kws={
                 'rwidth': 0.75,
                 'edgecolor': 'black',
                 'alpha': 0.3
             },
             color='blue',
             kde=True,
             ax=ax2,
             label='Blues')
ax2.legend(fontsize='x-large', title_fontsize='60',loc='upper left')

ax2.set_xlabel("Intensity value",fontsize=25)
ax2.set_ylabel("Density",fontsize=25)